In [325]:
import torch.nn.functional as F
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.parallel
# from PIL import Image
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [326]:
# 每次的个数
BATCH_SIZE = 20
# 迭代次数
EPOCHS = 8
# 采用cpu还是gpu进行计算
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [327]:
transform = transforms.Compose([
    transforms.Resize(100),
    transforms.RandomVerticalFlip(),
    transforms.RandomCrop(50),
    transforms.RandomResizedCrop(150),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
# 导入训练数据
dataset_train = datasets.ImageFolder('C:\\Users\\60553\\Desktop\\rgzn\\czxt', transform)

# 导入测试数据
dataset_test = datasets.ImageFolder('C:\\Users\\60553\\Desktop\\rgzn\\testdata', transform)

In [328]:
classess = dataset_train.classes  # 标签
class_to_idxes = dataset_train.class_to_idx  # 对应关系
print(class_to_idxes)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)

{'狗狗': 0, '鸟': 1}


# 六个卷积层，四个池化层和两个全连接层

In [329]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.max_pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.max_pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.max_pool3 = nn.MaxPool2d(2)
        self.conv5 = nn.Conv2d(64, 128, 3)
        self.conv6 = nn.Conv2d(128, 128, 3)
        self.max_pool4 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4608, 512)
        self.fc2 = nn.Linear(512, 1)

    def forward(self, x):
        in_size = x.size(0)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = F.relu(x)
        x = self.max_pool3(x)
        x = self.conv5(x)
        x = F.relu(x)
        x = self.conv6(x)
        x = F.relu(x)
        x = self.max_pool4(x)
        # 展开
        x = x.view(in_size, -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

In [330]:
model_lr = 0.0005
model = ConvNet().to(device)
print(model)
# 选择Adam优化器，学习率调低

optimizer = optim.Adam(model.parameters(), model_lr)


ConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv6): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (max_pool4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=1, bias=True)
)


In [331]:
def adjust_learning_rate(optimizer, epoch):
    modellrnew = model_lr * (0.1 ** (epoch // 5))
    print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew

In [332]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = data.to(device), target.to(device).float().unsqueeze(1)

        optimizer.zero_grad()

        output = model(data)

        # print(output)

        loss = F.binary_cross_entropy(output, target)

        loss.backward()

        optimizer.step()

        # if (batch_idx + 1) % 1 == 0:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
        #         100. * (batch_idx + 1) / len(train_loader), loss.item()))

In [333]:
def test(model, device, test_loader):
    model.eval()

    test_loss = 0

    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device).float().unsqueeze(1)
            # print(target)
            output = model(data)
            # print(output)
            test_loss += F.binary_cross_entropy(output, target, reduction='mean').item()
            pred = torch.tensor([[1] if num[0] >= 0.5 else [0] for num in output]).to(device)
            correct += pred.eq(target.long()).sum().item()

        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

In [334]:
for epoch in range(0, EPOCHS):
    adjust_learning_rate(optimizer, epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

lr: 0.0005

Test set: Average loss: 11.0377, Accuracy: 181/306 (59%)

lr: 0.0005

Test set: Average loss: 11.0661, Accuracy: 181/306 (59%)

lr: 0.0005

Test set: Average loss: 11.0670, Accuracy: 190/306 (62%)

lr: 0.0005

Test set: Average loss: 11.0578, Accuracy: 181/306 (59%)

lr: 0.0005

Test set: Average loss: 11.0683, Accuracy: 181/306 (59%)

lr: 5e-05

Test set: Average loss: 11.0700, Accuracy: 181/306 (59%)

lr: 5e-05

Test set: Average loss: 11.0735, Accuracy: 181/306 (59%)

lr: 5e-05

Test set: Average loss: 11.0762, Accuracy: 181/306 (59%)

lr: 5e-05

Test set: Average loss: 11.1001, Accuracy: 156/306 (51%)

lr: 5e-05

Test set: Average loss: 11.1034, Accuracy: 153/306 (50%)



In [335]:
torch.save(model, 'C:\\Users\\60553\\Desktop\\rgzn\\model\\model_insects.pth')